In [1]:
import numpy as np
import matplotlib.pyplot as plt

#Globally fix plot styling
import matplotlib as mpl
mpl.rc('xtick', direction='in', top=True)
mpl.rc('ytick', direction='in', right=True)
mpl.rc('xtick.minor', visible=True)
mpl.rc('ytick.minor', visible=True)

rng = np.random.default_rng()

# Average distance

This problem has been discussed multiple times online. One source is from [Mind Your Decisions](https://mindyourdecisions.com/blog/2016/07/03/distance-between-two-random-points-in-a-square-sunday-puzzle/) Presh Talwalkar shows how to analytically determine the average distance between two points in the unit square. This is hard to do analytically, but "trivial" to do numerically, ....

In [2]:
Nsamp = 10_000_000
Npts = 2
Ndim = 2
pts = rng.random(size=(Nsamp,Ndim,Npts))
diff = np.squeeze(np.diff(pts, axis=-1))
dist = np.sqrt(np.sum(diff**2, axis=-1))
np.mean(dist)

0.5216468970246964

In [3]:
Nrel = 100_000
Ndim = 3
Npts = 2
pts = rng.random(size=(Nrel, Ndim, Npts))
d = np.squeeze(np.diff(pts, axis=-1))
dist = np.sqrt(np.sum(d**2, axis=-1))
np.mean(dist)

0.6616407374450485

## Memory usage

It is easy to try to use too much memory!
We are storing floating point numbers which are 8 bytes. Thus we can calculate how much memory we are using.

In [4]:
def mem_usage(Nrel, Ndim, Npts=2):
    mem = Nrel * Ndim * Npts * 8 / 1024**3
    print(f'Using {mem} GB')

In [5]:
mem_usage(100_000_000, 10)

Using 14.901161193847656 GB


## General Solution

In [6]:
def average_distance_with_uncertainty(Nsets, Nrel, Ndim):
    Npts = 2 # It only makes sense to have 2 pts.
    pts = rng.random(size=(Nsets, Nrel, Ndim, Npts))
    diff = np.squeeze(np.diff(pts, axis=-1))
    dist = np.sqrt(np.sum(diff**2, axis=-1))
    avg = np.mean(dist, axis=-1)
    var = np.sqrt(np.var(avg))
    return np.mean(avg), var

In [7]:
Nrel = 100_000
ndimmax = 10
Nsets = 100
mem_usage(Nsets*Nrel, ndimmax)

Using 1.4901161193847656 GB


In [8]:
for ndim in range(2, ndimmax+1):
    print(ndim, average_distance_with_uncertainty(Nsets, Nrel, ndim))

2 (0.5215253497179162, 0.0007934338509869436)
3 (0.661790158076986, 0.0007333677516631766)
4 (0.7776312435912146, 0.000658835689265103)
5 (0.8784782522098402, 0.0007798873195964297)
6 (0.9690396501912268, 0.0007733155169881757)
7 (1.0516666908426529, 0.0006804780028018306)
8 (1.1282121381633008, 0.0008201592434864153)
9 (1.1998122035783558, 0.0007958783077582739)
10 (1.2675062279258646, 0.000683953795110894)


## Analytic Result

Analytically the answer for 2 dimensions is known to be
$$ \langle \mathrm{dist} \rangle = \frac{2 + \sqrt{2} + 5 \ln(\sqrt{2}+1)}{15}. $$

In [9]:
true_dist2 = (2 + np.sqrt(2) + 5 * np.log(np.sqrt(2) + 1)) / 15
true_dist2
dist2, err2 = average_distance_with_uncertainty(Nsets, Nrel, 2)
print(f'''Avg distance = {dist2} +/- {err2}
true distance = {true_dist2}
true error = {np.abs(true_dist2 - dist2)}''')

Avg distance = 0.5214832077171753 +/- 0.0007895894514202302
true distance = 0.5214054331647207
true error = 7.777455245461251e-05
